In [1]:
import pandas as pd

file_name=[
    '01','02','03','04','05','06','07','08','09','10',
    '11','12','13','14','15','16','17','18','19','20',
    '21','22','23','24','25','26','27','28','29','30',
    '31','32','33','34','35','36','37','38','39','40',
    '41','42','43','44','45','46','47','48','49','50',
    '51','52','53','54','55','56','57','58','59','60']

data=[]

for i in range(len(file_name)):
    data.append(pd.read_csv('../data/'+ file_name[i] +'_tracks.csv'))

# Standard deviation of speed

# Create Pull Request


In [46]:
# add standard deviation and add into the dataframe as a new column
import numpy as np
for i in range(len(file_name)):
    speed_sum=(data[i]['dhw']/data[i]['thw']).sum()
    speed_len=len(data[i]['dhw'])
    data[i]['speed_std']=np.sqrt(((data[i]['dhw']/data[i]['thw'])**2)-speed_sum/speed_len)

print(data[14]['speed_std'])

# DV1 


0               NaN
1               NaN
2               NaN
3               NaN
4               NaN
            ...    
311138    32.759997
311139    32.768660
311140    32.777322
311141    32.788872
311142    32.797535
Name: speed_std, Length: 311143, dtype: float64
